# Import the required libraries

In [1]:
# linear algebra
import numpy as np
# data processing, CSV file I/O (e.g. pd.read_csv)
import pandas as pd
# for dimensionality reduction
from sklearn.decomposition import PCA

# Read the data from train.csv

In [2]:
df_train = pd.read_csv('D:/dataset/train.csv')

In [3]:
# let us understand the data
print('Size of training set: {} rows and {} columns'
      .format(*df_train.shape))

Size of training set: 4209 rows and 378 columns


In [4]:
# print few rows and see how the data looks like
df_train.head()

,ID,y,X0,X1,X2,X3,X4,X5,X6,X8,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,0,130.81,k,v,at,a,d,u,j,o,...,0,0,1,0,0,0,0,0,0,0
1,6,88.53,k,t,av,e,d,y,l,o,...,1,0,0,0,0,0,0,0,0,0
2,7,76.26,az,w,n,c,d,x,j,x,...,0,0,0,0,0,0,1,0,0,0
3,9,80.62,az,t,n,f,d,x,l,e,...,0,0,0,0,0,0,0,0,0,0
4,13,78.02,az,v,n,f,d,h,d,n,...,0,0,0,0,0,0,0,0,0,0


# Collect the Y values into an array

In [5]:
# seperate the y from the data as we will use this to learn as 
# the prediction output
y_train = df_train['y'].values

# Understand the data types

In [6]:
# iterate through all the columns which has X in the name of the column
cols = [c for c in df_train.columns if 'X' in c]
print('Number of features: {}'.format(len(cols)))

print('Feature types:')
df_train[cols].dtypes.value_counts()

Number of features: 376
Feature types:


int64     368
object      8
dtype: int64

# Count the data in each of the columns

In [7]:
counts = [[], [], []]
for c in cols:
    typ = df_train[c].dtype
    uniq = len(np.unique(df_train[c]))
    if uniq == 1:
        counts[0].append(c)
    elif uniq == 2 and typ == np.int64:
        counts[1].append(c)
    else:
        counts[2].append(c)

print('Constant features: {} Binary features: {} Categorical features: {}\n'
      .format(*[len(c) for c in counts]))
print('Constant features:', counts[0])
print('Categorical features:', counts[2])

Constant features: 12 Binary features: 356 Categorical features: 8

Constant features: ['X11', 'X93', 'X107', 'X233', 'X235', 'X268', 'X289', 'X290', 'X293', 'X297', 'X330', 'X347']
Categorical features: ['X0', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X8']


# Read the test.csv data

In [8]:
df_test = pd.read_csv('D:/dataset/test.csv')

# remove columns ID and Y from the data as they are not used for learning
usable_columns = list(set(df_train.columns) - set(['ID', 'y']))
y_train = df_train['y'].values
id_test = df_test['ID'].values

x_train = df_train[usable_columns]
x_test = df_test[usable_columns]

#  Check for null and unique values for test and train sets

In [9]:
def check_missing_values(df):
    if df.isnull().any().any():
        print("There are missing values in the dataframe")
    else:
        print("There are no missing values in the dataframe")
check_missing_values(x_train)
check_missing_values(x_test)

There are no missing values in the dataframe
There are no missing values in the dataframe


# If for any column(s), the variance is equal to zero,then you need to remove those variable(s)

# Apply label encoder

In [10]:
for column in usable_columns:
    cardinality = len(np.unique(x_train[column]))
    if cardinality == 1:
        x_train.drop(column, axis=1) # Column with only one 
        # value is useless so we drop it
        x_test.drop(column, axis=1)
    if cardinality > 2: # Column is categorical
        mapper = lambda x: sum([ord(digit) for digit in x])
        x_train[column] = x_train[column].apply(mapper)
        x_test[column] = x_test[column].apply(mapper)
x_train.head()

C:\Users\suppy\anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
C:\Users\suppy\anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,X49,X294,X184,X237,X259,X12,X174,X4,X281,X164,...,X26,X163,X278,X282,X224,X218,X21,X181,X156,X182
0,0,0,1,1,0,0,0,100,0,0,...,0,0,0,0,0,0,1,0,1,0
1,0,0,0,0,0,0,0,100,0,0,...,0,0,0,0,0,0,0,0,1,0
2,0,0,0,0,0,0,0,100,0,0,...,0,1,0,0,1,1,0,0,0,0
3,0,0,0,0,0,0,1,100,0,0,...,0,0,0,0,0,1,0,0,0,0
4,0,0,0,0,0,0,0,100,0,0,...,0,0,0,0,0,1,0,0,0,0


# Make sure the data is now changed into numericals

In [11]:
print('Feature types:')
x_train[cols].dtypes.value_counts()

Feature types:


int64    376
dtype: int64

# Perform dimensionality reduction

In [12]:
# Linear dimensionality reduction using Singular Value Decomposition of 
# the data to project it to a lower dimensional space.
n_comp = 12
pca = PCA(n_components=n_comp, random_state=420)
pca2_results_train = pca.fit_transform(x_train)
pca2_results_test = pca.transform(x_test)

# Training using xgboost

In [13]:
import xgboost as xgb
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(
        pca2_results_train, 
        y_train, test_size=0.2, 
        random_state=4242)

d_train = xgb.DMatrix(x_train, label=y_train)
d_valid = xgb.DMatrix(x_valid, label=y_valid)
#d_test = xgb.DMatrix(x_test)
d_test = xgb.DMatrix(pca2_results_test)

params = {}
params['objective'] = 'reg:linear'
params['eta'] = 0.02
params['max_depth'] = 4

def xgb_r2_score(preds, dtrain):
    labels = dtrain.get_label()
    return 'r2', r2_score(labels, preds)

watchlist = [(d_train, 'train'), (d_valid, 'valid')]

clf = xgb.train(params, d_train, 
                1000, watchlist, early_stopping_rounds=50, 
                feval=xgb_r2_score, maximize=True, verbose_eval=10)

[13:50:26] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.1.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[0]	train-rmse:99.14835	valid-rmse:98.26297	train-r2:-58.35295	valid-r2:-67.63754
Multiple eval metrics have been passed: 'valid-r2' will be used for early stopping.

Will train until valid-r2 hasn't improved in 50 rounds.
[10]	train-rmse:81.27653	valid-rmse:80.36433	train-r2:-38.88428	valid-r2:-44.91014
[20]	train-rmse:66.71610	valid-rmse:65.77334	train-r2:-25.87403	valid-r2:-29.75260
[30]	train-rmse:54.86915	valid-rmse:53.89120	train-r2:-17.17724	valid-r2:-19.64513
[40]	train-rmse:45.24563	valid-rmse:44.22231	train-r2:-11.36018	valid-r2:-12.90160
[50]	train-rmse:37.44742	valid-rmse:36.37758	train-r2:-7.46672	valid-r2:-8.40697
[60]	train-rmse:31.15105	valid-rmse:30.01771	train-r2:-4.85891	valid-r2:-5.40526
[70]	train-rmse:26.08767	valid-rmse:24.90843	train-r2:-3.10906	valid-r2:-3.41037
[80]	train-rmse:22.04896	

# Predict your test_df values using xgboost

In [14]:
p_test = clf.predict(d_test)

sub = pd.DataFrame()
sub['ID'] = id_test
sub['y'] = p_test
sub.to_csv('xgb.csv', index=False)

sub.head()

,ID,y
0,1,82.714653
1,2,97.130867
2,3,83.082207
3,4,76.810875
4,5,112.846901
